<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/NER_Luigi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [17]:
#!pip3 install -U spacy[cuda92]
#!python3 -m spacy download en_core_web_lg
#https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon

import sys
import time
import os
import numpy as np
import random
import json
import spacy
from spacy import displacy
import nltk
import pprint
from nltk.tokenize import word_tokenize

nltk.download('punkt')

nlp = spacy.load("en_core_web_sm")
spacy.prefer_gpu()

from google.colab import drive

drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Language Analysis***

In [0]:

"""
1. X cite(s|d) Y1, Y2, …, Yn as (an) influence(s)
2. X was influenced by Y1, Y2, …, Yn
3. Y has been cited as an influence by X1, X2, …, Xn
4. Y influence on X1, X2, …, Xn …
5. Y1, Y2, …, Yn influenced hi(m|s) …
"""

def get_info(sent_list):
  for sent in sent_list:
    print(sent)
    doc = nlp(sent)
    displacy.render(doc, style="dep", jupyter=True)  
    print("CHUNKS")
        
    for chunk in doc.noun_chunks:
        print(chunk.text, chunk.root.text, chunk.root.dep_, 
                chunk.root.head.text)
    print("TREE")
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
                [child for child in token.children])


def process(phrase):
    influenced = []
    influencers = []
    doc = nlp(phrase)
    compounds = []
    for token in doc:
        if token.dep_ == "nsubjpass" and token.head.text in ["influenced", "influences"] :
            influenced.append(token.text)
        if token.dep_ == "nsubj" and token.head.text in ["influenced", "influences"]:
            influencers.append(token.text)
        if token.dep_ == "pobj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ in ["prep", "agent","conj"]:
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        if token.dep_ == "conj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ == "prep" or token.head.dep_ =="agent" or token.head.dep_ =="conj":
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        if token.dep_ == "dobj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ == "dobj" or token.head.dep_ == "prep" or token.head.dep_ =="agent":
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        
        if token.dep_ == "compound":
            compounds.append(token)
    
    for token in compounds:
        if token.head.text in influenced:
            influenced.append(token.text)
        if token.head.text in influencers:
            influencers.append(token.text)
    return influenced, influencers
    
phrases=["Mark was influenced by Jhon",
         "Jhon influenced Mark",
         "Mark was an influence of Jhon and Albert",
         "Jhon influenced him",
         "Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts ",
         "He is cited as an influence on John Milton"]

for phr in phrases:
  get_info([phr])
  print(process(phr))


#***Import JSON articles***


In [0]:
"""
{
    "philosopher": "name",
    "article": "plaintext_article",
    "pageid": "id",
    "table_influenced": 
        [
            {
                "0": "name_of_someone_philosopher_influenced_by"
            }
        ]
    "table_influences":
        [
            {
                "0": "name_of_someone_philosopher_influences"
            }
        ]
}
"""
#open the file
with open('/content/drive/My Drive/Wiki_dump/mattia_ground_t.json') as f:
  jsonlist = json.load(f)

phil_name_list = []

for art in jsonlist:
    phil_name_list.append(art['philosopher'])


# ***Text Preprocessing***

In [7]:
"""
#HIM
#divide in sentences the article

#eliminate sentences that don't include influenc -e, -ed, -es, -ing etc

#pattern matching on patterns (see language analysis)

#Entity recognition and recostruction

#US
"mark influenced jhon. Today is friday"
#divide in sentences the article
"mark influenced jhon"
"Today is friday"
#eliminate sentences that don't include influenc -e, -ed, -es, etc
"mark influenced jhon"

#pattern matching on patterns (using spacy 
?)
[x = "mark", y = "jhon"]

#Entity recognition and recostruction(here we have a proble, when he had Hendrix he could )
"""


import time
start = time.time()
counter_influence = []
counter_influenced = []
counter_influences = []
counter_influencing = []
counter_inspired = []
counter_inspire = []
counter_inspiration = []
for elem in jsonlist:
    #divide in sentences the article
    sent_list = nltk.sent_tokenize(elem["article"])
    
    sent_list = [word_tokenize(i) for i in sent_list]

    influence_list = []
    influence_declinations = ["influence", "influenced", "influences", "influencing", "inspired", "inspire", "inspiration"]


    for word_list in sent_list:
        temp = [x for x in word_list if x in influence_declinations]
        if "influence" in word_list:
            counter_influence.append(word_list)
        if "influencing" in word_list:
            counter_influencing.append(word_list)
        if "influenced" in word_list:
            counter_influenced.append(word_list)
        if "influences" in word_list:
            counter_influences.append(word_list)
        if "inspired" in word_list:
            counter_inspired.append(word_list)
        if "inspire" in word_list:
            counter_inspire.append(word_list)
        if "inspiration" in word_list:
            counter_inspiration.append(word_list)
        if len(temp) != 0:
            influence_list.append(' '.join(word for word in word_list))

    new_list = []
    for sent in influence_list:
        new_list.append(sent)
        #displacy.render(sent, style="dep", jupyter=True)
    elem["article"] = new_list
print(time.time() - start)


"""
influencing pisciato
inspire pisciato

influence only used as a name
influences only used as a name, plural of influences
inspiration only used as a name, similar to influence
influenced used as verb
inspired used like influenced

X's can be changed with his
Influence:
X be an influence (of/on/for/to) Y 
X's influence (of/on/for/to) Y
X be Y's influence

influences:
influences (include/of) X
X be Y's influences
X and X were influences of Y

influencing:
none

influenced
Y be influenced by X
X have influenced Y
Y's thinking was influenced by X
X influenced Y

X ispiratore Y ispirante
inspired
Y be influenced by X
X have inspired Y
Y had been inspired by X
Y's thinking/work was inspired by X
Y was inspired by X
X inspired Y

inspire
X seemed/(verb) to inspire Y
X's work inspire Y

inspiration
X became/was inspiration for Y
X's work provided/was inspiration for Y
X's work served as inspiration to Y
Y took/drew inspiration from X
X provided inspiration to Y
"""

print(len(counter_influence), "influence",
      len(counter_influences), "influences",
      len(counter_influencing), "influencing",
      len(counter_influenced), "influenced",
      len(counter_inspired), "inspired",
      len(counter_inspire), "inspire",
      len(counter_inspiration), "inspiration")
for wl in counter_inspired:
    print(' '.join(word for word in wl))

35.3787579536438
1749 influence 221 influences 35 influencing 1282 influenced 374 inspired 31 inspire 201 inspiration
Vidyaranya , also known as Madhava , who was the 12th Jagadguru of the Śringeri Śarada Pītham from 1380 to 1386 , inspired the re-creation of the Hindu Vijayanagara Empire of South India in response to the devastation caused by the Islamic Delhi Sultanate .
Then from 1952 to 1960 he inspired a group of scholars for a `` New Enlightenment , '' and organized a series of conventions attended by the philosophers who were engaged in the construction of a `` lay '' philosophy and who were interested in the main trends of the foreign philosophical thought .
John Trenchard and Thomas Gordon were inspired by the play to write an epistolary exchange entitled Cato 's Letters , concerning individual rights , using the name `` Cato '' .The action of the play involves the forces of Cato at Utica , awaiting the attack of Caesar immediately following his victory at Thapsus ( 46 BC ) .


# ***Processing Giuliano(Spacy Dependencies)***

In [9]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""



"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""
"""
import re
print(jsonlist_no_empty[20]["article"][0])
tokens = [i for i in jsonlist_no_empty[20]["article"][0] if i.isupper() ]
print(tokens)
"""

# for phil in jsonlist_no_empty:
#     display_info(phil["article"])


import time
start = time.time()

for elem in jsonlist:
    #divide in sentences the article
    sent_list = nltk.sent_tokenize(elem["article"])
    
    sent_list = [word_tokenize(i) for i in sent_list]

    influence_list = []
    influence_declinations = ["influence", "influenced", "influences", "influencing"]

    for word_list in sent_list:
        temp = [x for x in word_list if x in influence_declinations]
        if len(temp) != 0:
            influence_list.append(' '.join(word for word in word_list))

    new_list = []
    for sent in influence_list:
        new_list.append(sent)
        #displacy.render(sent, style="dep", jupyter=True)
    elem["article"] = new_list

print(time.time() - start)

34.77304148674011


# ***Processing Mattia(Regex)***

In [0]:
"""
#some prints and infos

pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""
import re
#pprint.pprint(jsonlist)
stringone = 'His philosophy is mainly influenced by such thinkers as Nietzsche , Epicurus , the Cynic and Cyrenaic schools , as well as French materialism .'
relations = re.findall(r'[A-Z][a-z]+[\s]', stringone)
influence = re.findall(r'[iI]nfluenced[\s]', stringone)
dio = re.findall(r'[A-Z][a-z]+[\s].*[iI]nfluenced[\s].*[A-Za-z]+[\s]', stringone)
#print(re.split('[iI]nfluenced[\s]by', stringone ,2)[1])
left = re.split('[iI]nfluenced[\s]by', stringone ,2)[0]
#left = re.findall(r'[A-Z][a-z]+[\s].*[iI]nfluenced[\s]', stringone)
right = re.split('[iI]nfluenced[\s]by', stringone ,2)[1]
#right = re.findall(r'[iI]nfluenced by[\s].*', stringone)
influencers = re.findall(r'[A-Z][a-z]+[\s]', str(right))
influenced = re.findall(r'[A-Z][a-z]+[\s]', str(left))
"""
print(stringone)
print('Left: ' + str(left))
print('Right: ' + str(right))
print('Influencers: ' + str(influencers))
print('Influenced: ' + str(influenced))
"""



jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]

for pippo in jsonlist_no_empty:
  #print(len(pippo['article']))
  #print(pippo.keys())
  pippo['influencers'] = []
  pippo['influenced'] = []
  pippo['rich_article'] = {}
  for i in range(len(pippo['article'])):
    left = []
    right = []
    influencers = []
    influenced = []

    #print('Sentence: ' + pippo['article'][i])
    try:
      left = re.split('have[\s].*[iI]nfluenced[\s]', pippo['article'][i] ,2)[0]
      right = re.split('have[\s].*[iI]nfluenced[\s]', pippo['article'][i] ,2)[1]
      influencers = re.findall(r'[A-Z][a-z]+[\s]', str(left))
      influenced = re.findall(r'[A-Z][a-z]+[\s]', str(right))
    except:
      try:
        left = re.split('was[\s].*[iI]nfluenced[\s]', pippo['article'][i] ,2)[0]
        right = re.split('was[\s].*[iI]nfluenced[\s]', pippo['article'][i] ,2)[1]
        influencers = re.findall(r'[A-Z][a-z]+[\s]', str(right))
        influenced = re.findall(r'[A-Z][a-z]+[\s]', str(left))
      except:
        try:
          left = re.split('[iI]nfluenced[\s]by[\s]', pippo['article'][i] ,2)[0]
          right = re.split('[iI]nfluenced[\s]by[\s]', pippo['article'][i] ,2)[1]
          influencers = re.findall(r'[A-Z][a-z]+[\s]', str(right))
          influenced = re.findall(r'[A-Z][a-z]+[\s]', str(left))
        except:
          try:
            left = re.split('has[\s][iI]nfluenced[\s]', pippo['article'][i] ,2)[0]
            right = re.split('has[\s][iI]nfluenced[\s]', pippo['article'][i] ,2)[1]
            influencers = re.findall(r'[A-Z][a-z]+[\s]', str(left))
            influenced = re.findall(r'[A-Z][a-z]+[\s]', str(right))
          except:
            try:
              left = re.split('[iI]nfluenced[\s]', pippo['article'][i] ,2)[0]
              right = re.split('[iI]nfluenced[\s]', pippo['article'][i] ,2)[1]
              influencers = re.findall(r'[A-Z][a-z]+[\s]', str(left))
              influenced = re.findall(r'[A-Z][a-z]+[\s]', str(right))
            except:
              try:
                left = re.split('[iI]nfluence[\s].*on[\s]', pippo['article'][i] ,2)[0]
                right = re.split('[iI]nfluence[\s].*on[\s]', pippo['article'][i] ,2)[1]
                influencers = re.findall(r'[A-Z][a-z]+[\s]', str(left))
                influenced = re.findall(r'[A-Z][a-z]+[\s]', str(right))
              except:
                try:
                  left = re.split('[iI]nfluence[\s].*of[\s]', pippo['article'][i] ,2)[0]
                  right = re.split('[iI]nfluence[\s].*of[\s]', pippo['article'][i] ,2)[1]
                  influencers = re.findall(r'[A-Z][a-z]+[\s]', str(right))
                  influenced = re.findall(r'[A-Z][a-z]+[\s]', str(left))
                except:
                  influencers = []
                  influenced = []
    pippo['rich_article'][i] = {}
    pippo['rich_article'][i]['phrase'] = pippo['article'][i]
    pippo['rich_article'][i]['influenced'] = influenced
    pippo['rich_article'][i]['influencers'] = influencers

    pippo['influencers'].append(influencers)
    pippo['influenced'].append(influenced)
    #print('Influencers: ' + str(influencers))
    #print('Influenced: ' + str(influenced))

for pippo in jsonlist_no_empty:
  pippo['influenced'] = [item for sublist in pippo['influenced'] for item in sublist]
  pippo['influencers'] = [item for sublist in pippo['influencers'] for item in sublist]
  pprint.pprint(pippo)

# ***Processing Luigi(Regex)***

In [61]:

# for phr in phrases_inspired:
#   get_info([phr[0]])
#   influenced, influencer = process(phr[0])
#   print("______EVAL")
#   print("influenced = ",phr[1], "influencers", phr[2])
#   print("influenced = ",influenced, "influencers", influencer)
#   print(len([x for x in influenced if x in phr[1]]), "on", len(phr[1]))
#   print(len([x for x in influencer if x in phr[2]]), "on", len(phr[2]))
"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" : "philosopher_name",
        "pageid" : id,
        "article" : 
            [
                "frase contenente influence",

                "frase contenente influence" 
            ]

        "rich_article" :
            {
                index: 
                {
                    "influenced" :
                        [
                            "list of influensed chunks"
                        ],
                    "Influencers" :
                        [
                            "list of influensers chunks"
                        ],
                    "phrase" : "frase contenente influence"
                }
            } 


                "frase contenente influence" = "Jhon Milton influenced Ghandi and Hannibal Bures' "Power to the Man""
                [
                    list of influencers = ["Jhon", "Milton"] --> "Jhon Milton"
                ],
                [
                    list of influenced = ["Ghandi", "Hannibal", "Bures", "Power", "Man"] --> "Mathma Ghandi", "Hannibal Bures"
                ]
                "him" --> nome del philosopher

"""

phrases_inspire=[
                    ["He seemed to inspire her",["her"],["He"]],
                    ["These revolutionizing ideas of Wang Yangming would later inspire prominent Japanese thinkers like Motoori Norinaga",["Mootori Norinaga"],["Wang Yangming"]],
                    ["John's work inspire Mark",["John"],["Mark"]]
                ]
phrases_inspiration=[
                        ["John was inspiration for Mark",["Mark"],["John"]],
                        ["Rozanov is the main source of inspiration for Dmitry Galkovsky",["Dmitry Galkovsky"],["Rozanov"]],
                        ["Jhon and Adam became inspiration for Mark",["Mark"],["John and Adam"]],
                        ["Jhon's work provided inspiration for Mark",["Mark"],["John"]],
                        ["He got the inspiration for this text from Schleiermacher ’ s Über die Religion ",["He"],["Schleiermacher"]],
                        ["Jhon's work was inspiration for Mark",["Mark"],["John"]],
                        ["While Murdoch 's thought is an inspiration for Conradi",["Conradi"],["Murdoch"]],
                        ["Jhon's work served as inspiration to Mark",["Mark"],["Jhon"]],
                        ["Lucian 's True Story inspired Cyrano de Bergerac , whose writings later served as inspiration for Jules Verne ",["Cyrano de Bergerac", "Jules Verne"],["Lucian"]],
                        ["Mark took inspiration from John",["Mark"],["John"]],
                        ["He also took inspiration from phenomenologist epistemology",["He"],["phenomenologist epistemology"]],
                        ["Mark drew inspiration from John", ["Mark"], ["John"]],
                        ["In particular , he drew inspiration from a Chinese Buddhist master named Tao-cho",["he"],["Tao-cho"]],
                        ["Mark provided inspiration to John",["John"],["Mark"]]
                    ]
phrases_inspired=[
                    ["He was inspired by him",["He"],["him"]],
                    ["Mark have inspired Jhon",["John"],["Mark"]],
                    ["Jhon had been inspired by Mark",["John"],["Mark"]],
                    ["In it , Petrarch claimed to have been inspired by Philip V of Macedon",["Petrarch"],["Philip V of Macedon"]],
                    ["Jhon's thinking was inspired by Mark",["John"],["Mark"]],
                    ["Newton 's work on infinite series was inspired by Simon Stevin 's decimals",["Newton"],["Simon Stevin"]],
                    ["Jhon's work was inspired by Marks",["John"],["Marks"]],
                    [ "Schiller was inspired by the play Julius of Tarent by Johann Anton Leisewitz .",["Schiller"],["Johann Anton Leisewitz"]],
                    [ "Mark was inspired by Jhon",["Mark"],["John"]],
                    ["As a youth , he was inspired by Mencius ’ proposition",["he"],["Mencius"]],
                    ["Jhon inspired Mark",["John"],["Mark"]],
                    [ "It also inspired him to take falsifiability as his criterion of demarcation between what is , and is not , genuinely scientific",["him"],["It"]],
                    ["Jhon inspired Mark's work",["Mark"],["John"]],
                    [ "Spinoza inspired the poet Shelley to write his essay",["Shelley"],["Spinoza"]]
                    
                  ]


output = [
          {'article': ['== Influence == Zhuangzi has influenced thinking far beyond East '
             'Asia .'],
 'influenced': ['East ', 'Asia '],
 'influencers': ['Influence ', 'Zhuangzi '],
 'pageid': '166152',
 'philosopher': 'Zhuang Zhou',
 'rich_article': {0: {'influenced': ['East ', 'Asia '],
                      'influencers': ['Influence ', 'Zhuangzi '],
                      'phrase': '== Influence == Zhuangzi has influenced '
                                'thinking far beyond East Asia .'}}},
          {'article': ['== Influence == Zhuangzi has influenced thinking far beyond East '
             'Asia .'],
 'influenced': ['East ', 'Asia '],
 'influencers': ['Influence ', 'Zhuangzi '],
 'pageid': '166152',
 'philosopher': 'Zhuang Zhou',
 'rich_article': {0: {'influenced': ['East ', 'Asia '],
                      'influencers': ['Influence ', 'Zhuangzi ',"Plato", "William", "Ralph", "Inge"],
                      'phrase': '== Influence == Zhuangzi has influenced '
                                'thinking far beyond East Asia .'}}}
]


def myfunc(phil):
    mtch_lst = []
    for p in phil_name_list:
        if p.startswith(phil) or phil in p:
            mtch_lst.append(p)
    return mtch_lst

def reconstruct_phil(output,phil_name_list):
    for match in output:
        art = match["rich_article"]
        for i in art:
            influenced = art[i]['influenced']
            influencers = art[i]['influencers']
            new_influenced = []
            new_influencers = []
            for phil in influencers:
                if phil in phil_name_list:
                    new_influenced.append(phil)
                    print("cioccato")
                else: #nome composto da piu stringhe ?
                    possible_names = myfunc(phil)
                    if len(possible_names) == 1: #cioccati ?
                        for name in possible_names:
                            splittato = name.split()
                            for part_of_name in splittato: #nome cognome
                                try:
                                    influencers.remove(part_of_name)
                                except:
                                    continue
                            new_influenced.append(name)
                    else:
                        print(possible_names)
    return new_influenced

print("RETURNED: ",reconstruct_phil(output,phil_name_list))
if "Elder" in phil_name_list:
    print("TRUE")


[]
[]
[]
[]
cioccato
['William Alston', 'William Blackstone', 'William Blake', 'Percy Williams Bridgman', 'William Ellery Channing', 'William Chillingworth', 'William Kingdon Clifford', 'William Crathorn', 'William K. Frankena', 'William Godwin', 'Sir William Hamilton, 9th Baronet', 'William Torrey Harris', 'William Harvey', 'William Hazlitt', 'William of Heytesbury', 'William Ernest Hocking', 'William Ralph Inge', 'William James', 'William Stanley Jevons', 'William Ernest Johnson', 'William Kneale (logician)', 'William Law', 'William Lycan', 'William McDougall (psychologist)', 'William Pepperell Montague', 'William of Ockham', 'William Paley', 'William David Ross', 'William S. Sahakian', 'William Ritchie Sorley', 'William Temple (archbishop)', 'William Thompson (Cork)', 'William Whewell', 'William of Alnwick', 'William of Auvergne, Bishop of Paris', 'William of Auxerre', 'William of Champeaux', 'William of Conches', 'William of Moerbeke', 'William of Shyreswood', 'Bernard Williams', '